# Extração de Características

Load

In [19]:
# imports
import numpy as np
# from math import prod

%run "at2-preprocessamento.ipynb"

print("\n====================== Importados da at2 ======================")

data, chunks_time, chunks_freq, Pxx = data, chunks_time, chunks_freq, Pxx #pra desbugar o lint

print(f'Data original {data.shape}')

voluntarios, classes_ensaios, canais, janelas, linhas = chunks_time.shape
chunks_time = chunks_time.reshape(voluntarios * classes_ensaios, canais, janelas, linhas)
print(f'Dominio do tempo: {chunks_time.shape} - (voluntario*classes*ensaios, canais, janelas, linhas)')

voluntarios, classes_ensaios, canais, janelas, linhas = chunks_freq.shape
chunks_freq = chunks_freq.reshape(voluntarios * classes_ensaios, canais, janelas, linhas)
print(f'Dominio da frequência:  {chunks_freq.shape} - (voluntario*classes*ensaios, canais, janelas, linhas)')

voluntário, classes_ensaios, canais, janelas = Pxx.shape
Pxx = Pxx.reshape(voluntarios * classes_ensaios, canais, janelas)
print(f'PSD-Welch (dom. da frequência): {Pxx.shape} - (voluntário*classes*ensaios, canais, janelas)')

10, 3, 8, 4, 1600
Formato (shape) dos dados depois da divisão de janelas
Dominio do tempo: (10, 24, 4, 33, 64) - (voluntario, classes+ensaios, canais, janelas, linhas)
Dominio da frequência:  (10, 24, 4, 33, 51) - (voluntario, classes+ensaios, canais, janelas, linhas)
Formato (shape) dos dados no domínio da frequência usando o método Welch:
Domínio da frequência: (10, 24, 4, 33) - (voluntário, classes+ensaios, canais, janelas)

====================== Importados da at2 ======================
Data original (10, 24, 4, 1600)
Dominio do tempo: (240, 4, 33, 64) - (voluntario*classes*ensaios, canais, janelas, linhas)
Dominio da frequência:  (240, 4, 33, 51) - (voluntario*classes*ensaios, canais, janelas, linhas)
PSD-Welch (dom. da frequência): (240, 4, 33) - (voluntário*classes*ensaios, canais, janelas)


### Funções de Extração de Características

In [48]:
# funções auxiliares
def PSD(w):
    ''' definição da função PSD para o sinal no domínio da frequência '''
    return np.abs(w) ** 2

def sgn(x):
    print(x)

### FUNÇÕES DE EXTRAÇÃO DE CARACTERÍSTICAS

# Dominio do Tempo
# WAMP, VAR, RMS, WL, ZC
def wamp(x):
    pass

def var(x):
    return np.sum(x ** 2, axis=-1) / (np.prod(x.shape) - 1)

def rms(x):
    return np.sqrt(np.sum(np.abs(x) ** 2, axis=-1) / (np.prod(x.shape) - 1))

def wl(x):
    pass

def zc(x):
    for i in x[:,:,:]:
        print(i)
    # np.sum(sgn(x), axis=-1)
    # pass

# Dominio da Frequencia
# FMD, FMN, MMDF, MMNF, PSD
def fmd(w):
    return np.sum(PSD(w), axis=-1) / 2

sample_rate = 200
def fmn(w):
    _,_,_,M = w.shape
    f_j = (np.arange(1, M+1) * sample_rate )/ (2 * M)  # Frequências correspondentes a cada índice
    weighted_sum = np.sum(f_j * PSD(w))
    mean_freq = weighted_sum / fmd(w)*2

    return mean_freq

def mmdf(w):
    return np.sum(np.abs(w), axis=-1) / 2

def mmnf(w):
    _,_,_,M = w.shape
    f_j = np.arange(1, M+1) * sample_rate / (2 * M) 
    weighted_sum = np.sum(f_j * np.abs(w), axis=-1)
    mean_freq = weighted_sum / mmdf(w)*2

    return mean_freq



#fmn(chunks_freq)
#mmnf(chunks_freq)



#PSD = Pxx ; welch
# f, Pxx = welch(data, fs=200, nperseg=64, noverlap=32)


array([[[215.9949514 , 217.62363877, 230.61637616, ..., 239.77985478,
         250.70277115, 246.62946621],
        [291.22634281, 289.58620635, 271.52593752, ..., 275.34526786,
         275.72548903, 267.02649322],
        [241.20371586, 242.86263965, 250.62890526, ..., 250.55453273,
         253.25536546, 252.88307347],
        [197.78473673, 197.28365711, 208.75823251, ..., 265.29125499,
         265.80245148, 253.44718977]],

       [[206.93782904, 207.65630838, 202.63029273, ..., 224.40333317,
         213.22021903, 199.91929935],
        [225.93335446, 230.1753536 , 253.7188359 , ..., 237.05860278,
         246.32218544, 247.96304139],
        [251.1938069 , 246.17737316, 218.88586439, ..., 215.21616207,
         215.4271627 , 219.28781869],
        [201.12738399, 201.36832151, 199.69649332, ..., 224.1001984 ,
         214.34616805, 215.08796572]],

       [[234.245988  , 235.28175277, 235.21553357, ..., 268.57930216,
         261.8758008 , 265.75916618],
        [312.7528215 , 3

In [30]:

def zc(x):
    vols, canais, janelas, linhas = x.shape

    data = np.empty((vols, canais, janelas))

    for v in range(vols):
        for c in range(canais):
            for j in range(janelas):
                data[v][c][j] = len(np.where(np.diff(np.sign(x[v][c][j])))[0])
    
    return data


zc(chunks_time)

array([[[ 0.,  0.,  0., ..., 17.,  9., 12.],
        [ 8., 10.,  6., ...,  4.,  0.,  0.],
        [29., 27., 31., ..., 33., 34., 32.],
        [ 0.,  0.,  0., ..., 16., 11., 10.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 4., 10.,  3., ...,  1.,  0.,  9.],
        [30., 19., 24., ...,  2.,  4., 10.],
        [ 2.,  3.,  2., ...,  8.,  0., 10.]],

       [[ 0.,  0.,  0., ...,  4.,  6.,  4.],
        [ 0.,  0.,  0., ...,  5., 12.,  0.],
        [ 0.,  0.,  0., ..., 15., 14., 18.],
        [ 0.,  0.,  0., ...,  6.,  6.,  7.]],

       ...,

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  2.],
        [13., 34., 36., ...,  2., 15.,  2.],
        [ 0.,  0.,  0., ...,  0.,  6.,  6.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 3.,  0., 19., ...,  0.,  0.,  0.],
        [17., 15.,  0., ..., 10., 20., 13.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  2.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.

### Geração do vetor de características

In [16]:
final_data = list()

#dominio do tempo
final_data.append(var(chunks_time))
final_data.append(rms(chunks_time))

#dominio da freq
final_data.append(fmd(chunks_freq))
final_data.append(mmdf(chunks_freq))
final_data.append(Pxx)

final = np.array(final_data)
final.shape

(5, 240, 4, 33)